In [2]:
import trimesh
import numpy as np
import plotly.graph_objects as go

# === โหลด STL ===
mesh = trimesh.load("repaired_print_demo90deg.stl")
if isinstance(mesh, trimesh.Scene):
    mesh = mesh.dump(concatenate=True)

# === พารามิเตอร์ ===
layer_height = 0.2
extrusion_multiplier = 0.045
z_min, z_max = mesh.bounds[0][2], mesh.bounds[1][2]
num_layers = int((z_max - z_min) / layer_height)
max_layers_to_show = 40  # เอาหลายชั้นรวมกัน

# === เตรียม Plotly figure ===
fig = go.Figure()

# === เพิ่มเส้น path ทีละชั้นแบบ 3D ===
for i in range(min(num_layers, max_layers_to_show)):
    z = z_min + i * layer_height + 0.01
    section = mesh.section(plane_origin=[0, 0, z], plane_normal=[0, 0, 1])
    if section is None:
        continue

    planar, tf = section.to_planar()

    for entity in planar.entities:
        if hasattr(entity, 'points'):
            coords_2d = planar.vertices[entity.points]
            if len(coords_2d) < 2:
                continue

            coords_3d = np.column_stack((coords_2d, np.full(len(coords_2d), z)))
            coords_3d = trimesh.transform_points(coords_3d, tf)  # แปลงกลับเป็น world coordinates

            fig.add_trace(go.Scatter3d(
                x=coords_3d[:, 0],
                y=coords_3d[:, 1],
                z=coords_3d[:, 2],
                mode='lines',
                line=dict(width=2),
                name=f"Layer {i}"
            ))

# === ปรับ layout ให้หมุน/ซูมได้ ===
fig.update_layout(
    title="3D Slice Preview (All Layers)",
    scene=dict(
        xaxis_title='X (mm)',
        yaxis_title='Y (mm)',
        zaxis_title='Z (mm)',
        aspectmode='data'
    ),
    width=800,
    height=700,
    showlegend=False
)

fig.show()

C:\Users\BMEi\AppData\Local\Temp\ipykernel_10660\439426807.py:27: DeprecationWarning:

DEPRECATED: replace `path.to_planar`->`path.to_2D), removal 1/1/2026



In [ ]:
pip install --upgrade -r requirements.txt